In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.svm import SVC


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [27]:
columns_to_be_added_as_features = ['Sex','Age','SibSp','Parch','Pclass','Fare','Embarked']
train_df = pd.read_csv('/kaggle/input/titanic/train.csv', usecols=columns_to_be_added_as_features + ['Survived'])
test_df_matcher = pd.read_csv('/kaggle/input/titanic/test.csv', usecols=columns_to_be_added_as_features + ['PassengerId'])
test_df = test_df_matcher[columns_to_be_added_as_features].copy()
print(test_df.head())


      Sex   Age  SibSp  Parch  Pclass     Fare Embarked
0    male  34.5      0      0       3   7.8292        Q
1  female  47.0      1      0       3   7.0000        S
2    male  62.0      0      0       2   9.6875        Q
3    male  27.0      0      0       3   8.6625        S
4  female  22.0      1      1       3  12.2875        S


In [28]:
print("Number of rows in training set: {}".format(len(train_df)))
print("Number of rows in test set: {}".format(len(test_df)))


Number of rows in training set: 891
Number of rows in test set: 418


In [29]:
for column_title in columns_to_be_added_as_features:
    if column_title in ['Embarked', "Sex"]:
        continue
    train_df.loc[:, column_title] = pd.to_numeric(train_df[column_title], downcast="float")
    test_df.loc[:, column_title] = pd.to_numeric(test_df[column_title], downcast="float")

train_df["Survived"] = pd.to_numeric(train_df["Survived"], downcast="float")

/tmp/ipykernel_32/3635651406.py:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train_df.loc[:, column_title] = pd.to_numeric(train_df[column_title], downcast="float")
/tmp/ipykernel_32/3635651406.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  test_df.loc[:, column_title] = pd.to_numeric(test_df[column_title], downcast="float")


In [30]:
train_df['Embarked'].replace('Q', 0,inplace=True)
train_df['Embarked'].replace('S', 1,inplace=True)
train_df['Embarked'].replace('C', 2,inplace=True)

test_df['Embarked'].replace('Q', 0,inplace=True)
test_df['Embarked'].replace('S', 1,inplace=True)
test_df['Embarked'].replace('C', 2,inplace=True)

train_df['Sex'].replace('male', 0,inplace=True)
train_df['Sex'].replace('female', 1,inplace=True)

test_df['Sex'].replace('male', 0,inplace=True)
test_df['Sex'].replace('female', 1,inplace=True)

print(train_df.head())


   Survived  Pclass  Sex   Age  SibSp  Parch       Fare  Embarked
0       0.0     3.0    0  22.0    1.0    0.0   7.250000       1.0
1       1.0     1.0    1  38.0    1.0    0.0  71.283302       2.0
2       1.0     3.0    1  26.0    0.0    0.0   7.925000       1.0
3       1.0     1.0    1  35.0    1.0    0.0  53.099998       1.0
4       0.0     3.0    0  35.0    0.0    0.0   8.050000       1.0


In [31]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

train_df = normalize(train_df)
test_df = normalize(test_df)

print(train_df.head())

   Survived  Pclass  Sex       Age  SibSp  Parch      Fare  Embarked
0       0.0     1.0  0.0  0.271174  0.125    0.0  0.014151       0.5
1       1.0     0.0  1.0  0.472229  0.125    0.0  0.139136       1.0
2       1.0     1.0  1.0  0.321438  0.000    0.0  0.015469       0.5
3       1.0     0.0  1.0  0.434531  0.125    0.0  0.103644       0.5
4       0.0     1.0  0.0  0.434531  0.000    0.0  0.015713       0.5


In [32]:
train_df = train_df.sample(frac=1).reset_index(drop=True)

In [33]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [34]:
validation_set_ratio = 0.2
validation_set_size = int(len(train_df)*validation_set_ratio)
training_set_size = len(train_df) - validation_set_size

print("Total set size: {}".format(len(train_df)))
print("Training set size: {}".format(training_set_size))
print("Validation set size: {}".format(validation_set_size))

from sklearn.model_selection import train_test_split

train, val = train_test_split(train_df, test_size=validation_set_ratio)

train_X = train[columns_to_be_added_as_features]
train_Label = train[['Survived']]

val_X = val[columns_to_be_added_as_features]
val_Label = val[['Survived']]

Total set size: 891
Training set size: 713
Validation set size: 178


In [35]:
SVM_KERNEL = "linear"
SVM_C = 10
SVM_GAMMA = 0.00001

svm_model = SVC(kernel = SVM_KERNEL, C = SVM_C, gamma = SVM_GAMMA)
svm_model.fit(train_X, train_Label)


/opt/conda/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=10, gamma=1e-05, kernel='linear')

In [36]:
from sklearn import metrics

y_pred = svm_model.predict(val_X)
print("Accuracy:",metrics.accuracy_score(val_Label, y_pred))


Accuracy: 0.7486033519553073


In [37]:
test_pred = svm_model.predict(test_df)


In [38]:
result = pd.DataFrame(test_df_matcher['PassengerId'])
print(result.head(10))


   PassengerId
0          892
1          893
2          894
3          895
4          896
5          897
6          898
7          899
8          900
9          901


In [39]:
result.insert(1, "Survived", test_pred, True)
result["Survived"] = pd.to_numeric(result["Survived"], downcast="integer")
print(result.head(10))


   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1
5          897         0
6          898         1
7          899         0
8          900         1
9          901         0


In [40]:
result.to_csv("out.csv", index=False)
